## Comparisons BNT _ KLUSTA


In [1]:
# the following two lines indicate that external functions are auto-reloaded as soon as they change. 
%load_ext autoreload
%autoreload 2
# Print statements 
from __future__ import print_function # Python 2.x

In [2]:
# General stuff:
import sys
import argparse
import os
import json
import numpy as np
import math
import psycopg2
import cPickle
import numpy as np
import pandas as pd
from datetime import date
from tqdm import tqdm_notebook

# Plotting:
from matplotlib import pyplot as plt
import seaborn as sns
import matplotlib.cm as cm
import matplotlib as mpl
%matplotlib inline

# External functions from subfolder /database_helpers. 
# as soon as you change something in there and press save, it will auto reload on next execution.
from database_helpers.psql_start import *
from database_helpers.create_tables import *
from database_helpers.write2tables import *
from postgres_analysis_helpers.general import *
from postgres_analysis_helpers.get_data import *

# register pickle type to retrieve binary data from database
psycopg2.extensions.register_type(psycopg2.extensions.new_type(psycopg2.BINARY.values, 'BINARY-PICKLE', cast_pickle))

Loaded analysis helpers: General
Loaded postgres_analysis_helpers -> general
Loaded postgres_analysis_helpers -> get_data


In [3]:
db_status = test_connect()
if db_status == False:
    print('Grrr... no database connection could be established.')
else:
    print('Yippiyeah! Database connection is established!')

Connecting to the PostgreSQL database...
Yippiyeah! Database connection is established!


### Make an empty dataframe and call the retrieval function

In [4]:
base_dataframe = pd.DataFrame() # create empty dataframe or feed in a base dataframe from before! 

In [5]:
psql = psql_neuroballs(base_dataframe)

61 animals across 8 experimenters found!
Total # of 5740 sessions (unique clusters: 11604)


In [6]:
psql.retrieve_data()

Retrieving data ... empty dataframe
498 entries retrieved.


In [78]:
data = psql.data()

In [40]:
data.iloc[9]

tetrode_no                                                                 5
animal_id                                                              T4743
n_drive_user                                                         davidcr
session_ts                                               2016-10-01 15:42:23
cluster_no                                                                 7
session_name                                                      01102016s1
spikes                     [[0.2200625, 35.1750095242, -28.1654467146], [...
spiket_tracking_session                time      speed  speed_filtered  c...
Name: 9, dtype: object

In [47]:
np.array(data.iloc[9].spikes)[:,0]

array([  2.20062500e-01,   6.63635417e+00,   2.23241875e+01,
         2.29001042e+01,   2.37467917e+01,   2.38773750e+01,
         2.43668750e+01,   2.53915000e+01,   2.54750000e+01,
         2.56043958e+01,   2.57101250e+01,   2.58079583e+01,
         2.99371250e+01,   3.17250625e+01,   3.40683958e+01,
         3.42977708e+01,   3.44854167e+01,   3.75926458e+01,
         3.95072708e+01,   4.00303958e+01,   4.04050000e+01,
         4.37248750e+01,   4.39093958e+01,   4.93638750e+01,
         5.15179167e+01,   5.23651875e+01,   5.31102917e+01,
         5.31432917e+01,   5.32390833e+01,   5.51501875e+01,
         5.56034583e+01,   6.19136667e+01,   6.47151250e+01,
         6.76625208e+01,   6.88602708e+01,   7.01034583e+01,
         7.36299167e+01,   7.38896250e+01,   7.44851875e+01,
         7.58127083e+01,   7.62246667e+01,   7.62733333e+01,
         7.83550000e+01,   7.98639375e+01,   8.10846875e+01,
         8.12641667e+01,   8.17182083e+01,   8.33282500e+01,
         8.36425625e+01,

In [46]:
data.iloc[9].spiket_tracking_session.time

0         0.220062
1         4.901375
2         6.636354
3        10.809458
4        13.985000
5        14.435250
6        17.931646
7        22.324188
8        22.900104
9        23.746792
10       23.877375
11       24.366875
12       25.391500
13       25.475000
14       25.604396
15       25.710125
16       25.807958
17       29.937125
18       31.725062
19       32.528687
20       34.068396
21       34.297771
22       34.485417
23       37.592646
24       39.507271
25       40.030396
26       40.405000
27       43.724875
28       43.909396
29       49.363875
          ...     
756    1496.839833
757    1508.388500
758    1512.085000
759    1514.695667
760    1516.145000
761    1516.261667
762    1516.458333
763    1517.105000
764    1531.903000
765    1532.141333
766    1533.444000
767    1537.672667
768    1538.606500
769    1545.777333
770    1546.636167
771    1549.176500
772    1550.677667
773    1560.866000
774    1561.276000
775    1564.699167
776    1566.803333
777    1567.

In [63]:
len(a),len(b)

(573, 786)

In [71]:
from itertools import islice

def findOverlap(a, b, rtol = 1e-05, atol = 1e-08):
    ovr_a = []
    ovr_b = []
    start_b = 0
    for i, ai in enumerate(a):
        for j, bj in islice(enumerate(b), start_b, None):
            if np.isclose(ai, bj, rtol=rtol, atol=atol, equal_nan=False):
                ovr_a.append(i)
                ovr_b.append(j)
            elif bj > ai: # (more than tolerance)
                break # all the rest will be farther away
                #print('fail!')
            else: # bj < ai (more than tolerance)
                start_b += 1 # ignore further tests of this item
                #print('fail (2)!')
    return (ovr_a, ovr_b)

In [72]:
ovr_a,ovr_b= findOverlap(a, b, rtol = 1e-05, atol = 1e-08)

In [97]:
diffs = []
for no,i in enumerate(xrange(len(data))):
    try:
        a = np.array(data.iloc[no].spikes)[:,0]
        b = data.iloc[no].spiket_tracking_session.time.values
        ovr_a,ovr_b= findOverlap(a, b, rtol = 1e-6, atol = 1e-10)
        diffs.append(len(ovr_a)-len(a))
        print('{}:   {},{},{}'.format(no,len(ovr_a),len(a),len(b)))
    except AttributeError as err: 
        continue
    except IndexError as err: 
        continue

8:   812,812,970
9:   572,573,786
10:   129,129,143
11:   10924,10957,15299
12:   1087,1087,1426
13:   750,750,999
14:   212,212,348
15:   835,835,1182
16:   301,301,323
17:   1452,1452,1578
18:   395,396,433
19:   779,779,872
20:   4149,4148,5015
21:   1208,1208,1483
22:   174,174,192
23:   428,426,458
24:   13778,13764,15201
25:   789,785,839
26:   2981,2981,3172
27:   306,306,336
28:   2253,2253,2497
29:   2005,2005,2352
30:   583,583,696
31:   4740,4745,5506
32:   507,509,566
33:   150,150,164
34:   310,311,338
35:   929,929,995
36:   602,602,647
37:   771,771,824
38:   374,374,411
39:   1019,1019,1150
40:   1943,1943,2106
41:   1261,1261,1553
42:   1704,1704,2151
43:   999,999,1286
44:   1149,1149,1306
45:   3712,3712,4162
46:   4431,4431,4984
47:   3337,3337,3650
48:   1782,1782,2143
49:   2194,2194,2450
50:   8821,8822,11109
51:   1534,1530,1787
52:   3376,3376,3761
53:   792,794,1087
55:   921,921,1042
56:   1200,1200,1494
57:   1189,1193,1409
58:   2656,2656,3513
59:   5486,54

483:   3365,3365,4570
485:   2802,2800,3603
486:   6468,6420,7739
490:   1392,1392,2154
491:   1897,1897,3139
492:   308,308,437
493:   302,302,357
494:   10308,10306,15830
495:   1204,1204,1823
496:   2873,2873,4399


In [96]:
diffs_old  = diffs

In [98]:
diffs_old

[0,
 -1,
 0,
 -33,
 0,
 0,
 0,
 0,
 0,
 0,
 -1,
 0,
 1,
 0,
 0,
 2,
 14,
 4,
 0,
 0,
 0,
 0,
 0,
 -5,
 -2,
 0,
 -1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -1,
 4,
 0,
 -2,
 0,
 0,
 -4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -1,
 -1,
 0,
 -3,
 0,
 0,
 -4,
 1,
 -1,
 0,
 19,
 0,
 -2,
 1,
 0,
 0,
 0,
 0,
 0,
 289,
 0,
 0,
 2,
 0,
 -3,
 0,
 0,
 4,
 0,
 7,
 -1,
 0]